# visualizing termination time versus domain size
Tim Tyree<br>
6.11.2025

In [1]:
import os,sys,pandas as pd,numpy as np

In [2]:
from lib import *

# define module

In [3]:
# parse_iolines
def parse_iolines_single(input_fn, printing=False):
    n_input=-9999
    n_output=-9999
    with open(input_fn) as f:
#         trgt1='Printing Inputs...\n'
#         trgt2='Printing Outputs...\n'
        trgt1='Printing Inputs:\n'
        trgt2='Printing Outputs:\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n
    return n_input,n_output

In [4]:
def parse_outputs_single_trial(input_fn, n_output):
    """
    Example Usage:
TermTime = parse_outputs_single_trial(input_fn, n_output)
    """
    with open(input_fn) as f:
        for n,line in enumerate(f):
            if n == n_output+1:
                lin0 =line
                TermTime = eval(lin0[:-2].split('=')[-1])
                return TermTime
#         if n == n_output+1:
#             line_N = line
#         if n == n_output+2:
#             line_CollTime =line
# N_values=np.array([eval(s) for s in line_N[:-2].split(',')])
# CollTime_values=np.array([eval(s) for s in line_CollTime[:-2].split(',')])
# # CollTime_values=np.array([eval(s) for s in line_CollTime[:-1].split(',')])
# df=pd.DataFrame({
#     'N':N_values,
#     'CollTime':CollTime_values
# })
# df=df.loc[df.CollTime!=-9999]
# df.reset_index(inplace=True,drop=True)
## return df

In [5]:
# parse_outputs(input_fn, n_output=)
def parse_log_single(input_fn,printing=False):
    """
    Example Usage:
dict_data = parse_log_single(input_fn,printing=False)
    """
    n_input,n_output=parse_iolines_single(input_fn, printing=False)
    if n_input==-9999:
        if printing:
            print(f"Warning: n_input not found for input_fn={input_fn}.\nreturning None...")
        return None
    col_name_lst, col_value_lst=parse_inputs(input_fn,n_input,n_output,printing=printing)
    dict_inputs=dict(zip(col_name_lst,col_value_lst))
    TermTime = parse_outputs_single_trial(input_fn, n_output)
    dict_data = dict(dict_inputs)
    dict_data['TermTime'] = TermTime
    return dict_data

# dev

In [6]:
run_num = 48
# fldr = f"{nb_dir}/data/osg_output/run_44"
# fldr = f"{nb_dir}/data/osg_output/run_45"
# fldr = f"{nb_dir}/data/osg_output/run_46"
# fldr = f"{nb_dir}/data/osg_output/run_47"
fldr = f"{nb_dir}/data/osg_output/run_{run_num}"

In [7]:
assert os.path.exists(fldr)

In [8]:
dict_data_lst = []
for fn in os.listdir(fldr):
    if fn.find('.out.')!=-1:
        input_fn = os.path.join(fldr,fn)
        dict_data = parse_log_single(input_fn,printing=False)
        if dict_data is not None:
            #record
            dict_data_lst.append(dict_data)
df=pd.DataFrame(dict_data_lst); del dict_data_lst

In [9]:
# np.array([x for x in dict_data_lst if x is None])

In [10]:
# pd.DataFrame(dict_data_lst)

In [11]:
# len(dict_data_lst)

In [12]:
# input_fn

In [13]:
# !atom '/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_45/job.out.17477944.1912'

In [14]:
# dict_data = parse_log_single(input_fn,printing=True)
# dict_data

In [17]:
quantile = 0.99
np.random.seed(42)
printing=True
if printing:
    print(f"LPM with Same Creation (Run #48):")
dict_lst = []
for (D,L),g in df.groupby(by=['D','L']):
    tau_values = g['TermTime'].values
    thresh = np.quantile(tau_values,quantile)
    tau_values = tau_values[tau_values<thresh].copy()
    meanx,Delta_meanx,num_obs,p_normal=comp_mean_bootstrap_uncertainty(tau_values)
    printing=True
    a=g['varkappa'].max()
    chi=g['chi'].max()
    if printing:
        print(f"- {D=}, {L=}: mean tau: {meanx:.4f} +/- {Delta_meanx:.4f} seconds (N={num_obs}, {p_normal=:.4f})")
    dict_lst.append(dict(D=D,L=L,a=a,chi=chi,
                         tau=meanx,Delta_tau=Delta_meanx,
                         num_obs=num_obs,p_normal=p_normal,
                        ))

LPM with Same Creation (Run #48):
- D=0.115, L=3.75: mean tau: 2.4068 +/- 0.0997 seconds (N=1980, p_normal=0.0426)
- D=0.115, L=4.5: mean tau: 8.8592 +/- 0.3686 seconds (N=1980, p_normal=0.8422)
- D=0.115, L=4.75: mean tau: 15.1341 +/- 0.6518 seconds (N=1980, p_normal=0.1434)
- D=0.115, L=5.0: mean tau: 25.0676 +/- 1.0480 seconds (N=1980, p_normal=0.5439)
- D=0.115, L=5.25: mean tau: 44.6624 +/- 1.8434 seconds (N=1980, p_normal=0.3428)
- D=0.115, L=5.625: mean tau: 107.9789 +/- 8.4754 seconds (N=1980, p_normal=0.0083)
- D=0.115, L=6.25: mean tau: 463.4994 +/- 27.5521 seconds (N=1942, p_normal=0.7934)
- D=0.115, L=7.075: mean tau: 1786.5926 +/- 72.9578 seconds (N=1612, p_normal=0.4804)
- D=0.115, L=8.75: mean tau: 2922.8952 +/- 236.9993 seconds (N=223, p_normal=0.1863)
- D=0.42, L=5.0: mean tau: 0.5154 +/- 0.0215 seconds (N=1980, p_normal=0.7469)
- D=0.42, L=6.25: mean tau: 1.5493 +/- 0.0662 seconds (N=1980, p_normal=0.2544)
- D=0.42, L=7.5: mean tau: 5.6377 +/- 0.2442 seconds (N=1980, 

LPM with Same Creation (Run #48) quantile=1.:
- D=0.115, L=3.75: mean tau: 2.5173 +/- 0.1073 seconds (N=2000, p_normal=0.5600)
- D=0.115, L=4.5: mean tau: 14.5283 +/- 7.3355 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=4.75: mean tau: 29.2283 +/- 11.8199 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=5.0: mean tau: 53.8336 +/- 17.2470 seconds (N=2000, p_normal=0.0000)
- D=0.115, L=5.25: mean tau: 70.9511 +/- 15.6924 seconds (N=2000, p_normal=0.0552)
- D=0.115, L=5.625: mean tau: 166.4115 +/- 27.6704 seconds (N=2000, p_normal=0.0342)
- D=0.115, L=6.25: mean tau: 624.0579 +/- 49.0198 seconds (N=2000, p_normal=0.4734)
- D=0.115, L=7.075: mean tau: 2603.9936 +/- 96.3658 seconds (N=2000, p_normal=0.9836)
- D=0.115, L=8.75: mean tau: 5656.2152 +/- 49.9363 seconds (N=1996, p_normal=0.1367)
- D=0.42, L=5.0: mean tau: 1.6145 +/- 2.1266 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=6.25: mean tau: 2.0326 +/- 0.8127 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=7.5: mean tau: 15.9232 +/- 7.1594 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=8.75: mean tau: 36.2174 +/- 7.2507 seconds (N=2000, p_normal=0.0000)
- D=0.42, L=10.0: mean tau: 171.8601 +/- 18.8753 seconds (N=2000, p_normal=0.0761)

In [18]:
df_out = pd.DataFrame(dict_lst)
df_out

,D,L,a,chi,tau,Delta_tau,num_obs,p_normal
0,0.115,3.750,1.552,2.939433,2.406796,0.099690,1980,0.042606
1,0.115,4.500,1.552,2.939433,8.859208,0.368588,1980,0.842247
2,0.115,4.750,1.552,2.939433,15.134093,0.651765,1980,0.143401
3,0.115,5.000,1.552,2.939433,25.067592,1.048049,1980,0.543925
4,0.115,5.250,1.552,2.939433,44.662364,1.843427,1980,0.342751
5,0.115,5.625,1.552,2.939433,107.978882,8.475434,1980,0.008281
6,0.115,6.250,1.552,2.939433,463.499385,27.552122,1942,0.793398
7,0.115,7.075,1.552,2.939433,1786.592594,72.957845,1612,0.480361
8,0.115,8.750,1.552,2.939433,2922.895171,236.999272,223,0.186273
9,0.420,5.000,9.300,2.075269,0.515361,0.021465,1980,0.746905


In [20]:
# savedir = f"{nb_dir}/data/osg_output/run_46_tau_vs_L.csv"
# savedir = f"{nb_dir}/data/osg_output/run_47_tau_vs_L.csv"
savedir = f"{nb_dir}/data/osg_output/run_{run_num}_tau_vs_L_quantile_.99.csv"

In [21]:
df_out.to_csv(savedir,index=False)
print(savedir)

/Users/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/run_48_tau_vs_L_quantile_.99.csv


In [22]:
beep(3)

-n 
-n 
-n 


0